In [2]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
import requests

In [3]:
# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [4]:
print(add.invoke({"a": 2, "b": 3}))
print(multiply.invoke({"a": 2, "b": 3}))

5
6


In [5]:
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)

In [6]:
llm_with_tools = llm.bind_tools([add, multiply])

In [7]:
result = llm_with_tools.invoke("What is 2 + 3")

In [8]:
result.tool_calls[0]

{'name': 'add',
 'args': {'a': 2, 'b': 3},
 'id': 'fc_9f50b226-0b96-4b10-9912-a6f9347e3802',
 'type': 'tool_call'}

In [9]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='fc_9f50b226-0b96-4b10-9912-a6f9347e3802')

In [10]:
query = HumanMessage('can you multiply 3 with 1000')
messages = [query]
result = llm_with_tools.invoke(messages)

In [11]:
messages.append(result)

In [12]:
tool_result = multiply.invoke(result.tool_calls[0])

In [13]:
messages.append(tool_result)

In [14]:
llm_with_tools.invoke(messages).content

'Sure! 3 × 1000 = **3000**.'